## Import the libraries

In [1]:
# For data handling and EDA
import pandas as pd
import numpy as np
# For Visualisation
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from wordcloud import STOPWORDS
# For NLP and ML
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, f1_score
from sklearn.svm import LinearSVC
import nltk
import spacy
import keras
from keras.preprocessing.text import Tokenizer

## Using the whole dataset

### Load the whole dataset to Pandas

In [ ]:
# Load the annotations metadata csv to get label info
annot = pd.read_csv("hate-speech-dataset-master/annotations_metadata.csv")
# Keep only file id and label
annot.drop(columns=['user_id','subforum_id','num_contexts'],inplace=True)
# Add new empty column to insert the message
annot['message']=''
# Loop through all lines
for i in range(len(annot['file_id'])):
    # Get current filename
    filename = annot['file_id'][i]
    # Open the file and read the contents
    with open('hate-speech-dataset-master/all_files/' +filename +'.txt') as f:
        contents= f.read()
    f.close()
    # Add contents to the message column
    annot['message'][i]=contents
# Drop file id
annot.drop(columns=['file_id'],inplace=True)

### EDA and Pre-Processing

In [ ]:
# DataFrame information
annot.info()

In [ ]:
# Check the classes and their value counts
annot['label'].value_counts()

Based on the original paper, there are a total of 10944 sentences, 9507 of them are classified as no hate speech, 1196 are classified as hate speech, 168 are related to hate speech but depends on content and 73 are non English sentences, classified as idk/skip. For the purpose of this research, relation and idk/skip classes will be dropped completely in order to make the problem a binary classification.

In [ ]:
# Keep sentences classified only as noHate and hate
cleanPosts = annot[(annot['label'] != 'relation') & (annot['label'] != 'idk/skip')]
# Reset index
cleanPosts=cleanPosts.reset_index(drop=True)

In [ ]:
# First 5 data points
cleanPosts.head(5)

In [ ]:
# Last 5 data points
cleanPosts.tail(5)

In [ ]:
# Update STOPWORDS
stop_words = STOPWORDS.update(["https", "S"])

In [ ]:
# WordCloud for posts classified as hate
wc_hate = WordCloud(stopwords=STOPWORDS, background_color='white').generate(' '.join(cleanPosts['message'][cleanPosts['label']=='hate']))
# Generate plot
plt.imshow(wc_hate)
plt.axis("off")
plt.show()

In [ ]:
# WordCloud for posts classified as noHate
wc_noHate = WordCloud(stopwords=STOPWORDS, background_color='white').generate(' '.join(cleanPosts['message'][cleanPosts['label']=='noHate']))
# Generate plot
plt.imshow(wc_noHate)
plt.axis("off")
plt.show()

### Recreating Paper Results

#### Pre-Process text with SpaCy

Before I proceed with the Bag of Words model, I need to pre-process the text (remove punctuations, stop words, lemmatize) in order to reduce the vocabulary size. I will use SpaCy for text pre-processing.

In [ ]:
# Get English words from NLTK
#nltk.download('words')
#words = set(nltk.corpus.words.words())
# Instantiate SpaCy's English module
nlp = spacy.load('en_core_web_sm')
# Remove default stop words and lemmatize each sentence
cleanPosts['clean_message'] = cleanPosts.message.apply(
    lambda text: 
        " ".join(
            token.lemma_.lower() for token in nlp(text)
                if token.lemma_.lower() not in nlp.Defaults.stop_words and token.is_alpha
                 #and token.lemma_.lower() in words
        )
)
# Drop old sentences
cleanPosts.drop(columns='message', inplace=True)

In [ ]:
# First 5 data points
cleanPosts.head(5)

In [ ]:
# Last 5 data points
cleanPosts.tail(5)

In [ ]:
# Check for empty strings and remove
cleanPosts[cleanPosts['clean_message'] == ''].count()

There are 233 empty entries after lemmatization and removing stop words, which I am going to remove.

In [ ]:
# Drop rows where 'clean_message' is an empty string
# Get index
idx = cleanPosts[cleanPosts['clean_message'] == ''].index
# Drop rows based on index
cleanPosts.drop(index=idx, inplace=True)

In [ ]:
# WordCloud for cleaned posts classified as hate
wc_hate_clean = WordCloud(stopwords=STOPWORDS, background_color='white').generate(' '.join(cleanPosts['clean_message'][cleanPosts['label']=='hate']))
# Generate plot
plt.imshow(wc_hate_clean)
plt.axis("off")
plt.show()

In [ ]:
# WordCloud for cleaned posts classified as noHate
wc_noHate_clean = WordCloud(stopwords=STOPWORDS, background_color='white').generate(' '.join(cleanPosts['clean_message'][cleanPosts['label']=='noHate']))
# Generate plot
plt.imshow(wc_noHate_clean)
plt.axis("off")
plt.show()

In [ ]:
# Plot the class distribution
plot = cleanPosts.label.value_counts().plot(kind='pie',autopct='%1.2f%%')

#### Bag-of-Words Vectors

In [ ]:
# Instantiate Count Vectorizer
vectorizer = CountVectorizer()
# Vectorize
X = vectorizer.fit_transform(cleanPosts.clean_message.to_list())
X_bow = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())

#### Split Dataset for SVM Model

In [ ]:
# Split dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_bow, cleanPosts.label, test_size=0.25, random_state=42, stratify=cleanPosts.label)

#### Classify text using SVM Model

Train the SVM Model

In [ ]:
# Instatiate SVC
svm = LinearSVC()
# Fit training data
svm.fit(X_train, y_train)
# Get prediction on test data
pred = svm.predict(X_test)

#### Get Accuracy and Confusion Matrix

In [ ]:
# Get the accuracy
print('Test Accuracy: '+str(svm.score(X_test, y_test)))
# Print the F1 Score
print('F1 Score: '+str(f1_score(y_test, pred, pos_label='hate')))
# Create confusion matrix
cm = confusion_matrix(y_test, pred, labels=svm.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=svm.classes_)
disp.plot()
plt.show()

Using all the available txt files with text pre-processing on a SVM Model returned an accuracy of 0.87 but an F1 Score of 0.48. This is because there is a big class inbalance in the training data. The SVM model does a really good job at classifying correctly the noHate sentences but handles poorly the hate sentences since more than half of them are missclassified.

## Using the sampled train and test dataset

### Load the provided sampled_train and sampled_test dataset

In [2]:
# Sampled Training Data
# Load the annotations metadata csv to get label info
annot_tr = pd.read_csv("hate-speech-dataset-master/annotations_metadata.csv")
# Keep only file id and label
annot_tr.drop(columns=['user_id','subforum_id','num_contexts'],inplace=True)
# Add new empty column to insert the message
annot_tr['message']=''
# Loop through all lines
for i in range(len(annot_tr['file_id'])):
    # Get current filename
    filename = annot_tr['file_id'][i]
    try:
        # Open the file and read the contents
        with open('hate-speech-dataset-master/sampled_train/' +filename +'.txt') as f:
            contents= f.read()
        f.close()
        # Add contents to the message column
        annot_tr['message'][i]=contents
    except:
        print('File '+str(filename)+' not in train samples.')
# Drop file id
annot_tr.drop(columns=['file_id'],inplace=True)

# Sampled Test Data
# Load the annotations metadata csv to get label info
annot_ts = pd.read_csv("hate-speech-dataset-master/annotations_metadata.csv")
# Keep only file id and label
annot_ts.drop(columns=['user_id','subforum_id','num_contexts'],inplace=True)
# Add new empty column to insert the message
annot_ts['message']=''
# Loop through all lines
for i in range(len(annot_ts['file_id'])):
    # Get current filename
    filename = annot_ts['file_id'][i]
    try:
        # Open the file and read the contents
        with open('hate-speech-dataset-master/sampled_test/' +filename +'.txt') as f:
            contents= f.read()
        f.close()
        # Add contents to the message column
        annot_ts['message'][i]=contents
    except:
        print('File '+str(filename)+' not in test samples.')
# Drop file id
annot_ts.drop(columns=['file_id'],inplace=True)

File 12834217_1 not in train samples.
File 12834217_3 not in train samples.
File 12834217_5 not in train samples.
File 12834217_6 not in train samples.
File 12834217_7 not in train samples.
File 12834217_8 not in train samples.
File 12834217_9 not in train samples.
File 12834217_10 not in train samples.
File 12843221_1 not in train samples.
File 12843221_2 not in train samples.
File 12845600_1 not in train samples.
File 12845600_2 not in train samples.
File 12845600_3 not in train samples.
File 12845600_4 not in train samples.
File 12845808_1 not in train samples.
File 12845808_2 not in train samples.
File 12849464_1 not in train samples.
File 12849464_3 not in train samples.
File 12849464_4 not in train samples.
File 12851329_1 not in train samples.
File 12851329_2 not in train samples.
File 12851329_3 not in train samples.
File 12854837_1 not in train samples.
File 12854837_2 not in train samples.
File 12854837_3 not in train samples.
File 12854837_4 not in train samples.
File 128553

### EDA and Pre-Processing

In [3]:
# Keep sentences classified only as noHate and hate - Train Sample
cleanPosts_tr = annot_tr[(annot_tr['label'] != 'relation') & (annot_tr['label'] != 'idk/skip')]
# Reset index
cleanPosts_tr = cleanPosts_tr.reset_index(drop=True)

# Keep sentences classified only as noHate and hate - Test Sample
cleanPosts_ts = annot_ts[(annot_ts['label'] != 'relation') & (annot_ts['label'] != 'idk/skip')]
# Reset index
cleanPosts_ts = cleanPosts_ts.reset_index(drop=True)

#### WordCloud for Posts Classified as Hate

In [ ]:
# Update STOPWORDS
stop_words = STOPWORDS.update(["https", "S"])

Train Sample

In [ ]:
# WordCloud for posts classified as hate - Train Sample
wc_hate_tr = WordCloud(stopwords=STOPWORDS, background_color='white').generate(' '.join(cleanPosts_tr['message'][cleanPosts_tr['label']=='hate']))
# Generate plot
plt.imshow(wc_hate_tr)
plt.axis("off")
plt.show()

Test Sample

In [ ]:
# WordCloud for posts classified as hate - Test Sample
wc_hate_ts = WordCloud(stopwords=STOPWORDS, background_color='white').generate(' '.join(cleanPosts_ts['message'][cleanPosts_ts['label']=='hate']))
# Generate plot
plt.imshow(wc_hate_ts)
plt.axis("off")
plt.show()

#### WordCloud for Posts Classified as noHate

Train Sample

In [ ]:
# WordCloud for posts classified as noHate - Train Sample
wc_noHate_tr = WordCloud(stopwords=STOPWORDS, background_color='white').generate(' '.join(cleanPosts_tr['message'][cleanPosts_tr['label']=='noHate']))
# Generate plot
plt.imshow(wc_noHate_tr)
plt.axis("off")
plt.show()

Test Sample

In [ ]:
# WordCloud for posts classified as noHate - Test Sample
wc_noHate_ts = WordCloud(stopwords=STOPWORDS, background_color='white').generate(' '.join(cleanPosts_ts['message'][cleanPosts_ts['label']=='noHate']))
# Generate plot
plt.imshow(wc_noHate_ts)
plt.axis("off")
plt.show()

### Recreating Paper Results

#### Pre-Process text with SpaCy

Train Sample

In [4]:
# Instantiate SpaCy's English module - Train Sample
nlp = spacy.load('en_core_web_sm')
# Remove default stop words and lemmatize each sentence
cleanPosts_tr['clean_message'] = cleanPosts_tr.message.apply(
    lambda text: 
        " ".join(
            token.lemma_.lower() for token in nlp(text)
                if token.lemma_.lower() not in nlp.Defaults.stop_words and token.is_alpha
        )
)
# Drop old sentences
cleanPosts_tr.drop(columns='message', inplace=True)

Test Sample

In [5]:
# Instantiate SpaCy's English module - Test Sample
nlp = spacy.load('en_core_web_sm')
# Remove default stop words and lemmatize each sentence
cleanPosts_ts['clean_message'] = cleanPosts_ts.message.apply(
    lambda text: 
        " ".join(
            token.lemma_.lower() for token in nlp(text)
                if token.lemma_.lower() not in nlp.Defaults.stop_words and token.is_alpha
        )
)
# Drop old sentences
cleanPosts_ts.drop(columns='message', inplace=True)

#### Drop rows with empty strings

Train Sample

In [6]:
# Drop rows where 'clean_message' is an empty string - Train Sample
# Get index
idx = cleanPosts_tr[cleanPosts_tr['clean_message'] == ''].index
# Drop rows based on index
cleanPosts_tr.drop(index=idx, inplace=True)

Test Sample

In [7]:
# Drop rows where 'clean_message' is an empty string - Train Sample
# Get index
idx = cleanPosts_ts[cleanPosts_ts['clean_message'] == ''].index
# Drop rows based on index
cleanPosts_ts.drop(index=idx, inplace=True)

#### Plot class distribution

In [ ]:
# Plot the class distribution - Train
plot_tr = cleanPosts_tr.label.value_counts().plot(kind='pie',autopct='%1.2f%%')

In [ ]:
# Plot the class distribution - Test
plot_ts = cleanPosts_ts.label.value_counts().plot(kind='pie',autopct='%1.2f%%')

#### Combine train and test samples into one pandas DataFrame

In [8]:
# Combine the train and test samples to use for the BoW vocabulary
cleanPosts_cb = pd.concat([cleanPosts_tr, cleanPosts_ts])

#### Bag-of-Words Vectors

In [ ]:
# Instantiate Count Vectorizer
vectorizer = CountVectorizer()
# Create the vocabulary
vectorizer.fit(cleanPosts_cb.clean_message.to_list())

# Transform train sample to document-term matrix
X_tr = vectorizer.transform(cleanPosts_tr.clean_message.to_list())
# Create a dataframe from the document-term matrix
X_train_b = pd.DataFrame(X_tr.toarray(),columns=vectorizer.get_feature_names_out())

# Transform test sample to document-term matrix
X_ts = vectorizer.transform(cleanPosts_ts.clean_message.to_list())
# Create a dataframe from the document-term matrix
X_test_b = pd.DataFrame(X_ts.toarray(),columns=vectorizer.get_feature_names_out())

#### Classify text using the SVM Model

Train the SVM Model

In [ ]:
# Instatiate SVC
svm = LinearSVC()
# Fit training data
svm.fit(X_train_b, cleanPosts_tr.label)
# Get prediction on test data
pred = svm.predict(X_test_b)

#### Get Accuracy, F1 Score and Confusion Matrix

In [ ]:
# Print the accuracy
print('Accuracy: '+str(svm.score(X_test_b, cleanPosts_ts.label)))
# Print the F1 Score
print('F1 Score: '+str(f1_score(cleanPosts_ts.label, pred, pos_label='hate')))
# Create confusion matrix
cm = confusion_matrix(cleanPosts_ts.label, pred, labels=svm.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=svm.classes_)
disp.plot()
plt.show()

Using the sampled train and test data returned an accuracy of 0.74, which is lower than the accuracy I got using the whole dataset. However, the F1 Score is much higher, 0.74, and the model does a fair job in correctly classifying both hate and noHate classes. These results are very close to the results from the paper and indicate the importance of having a class balaned dataset when it comes to text classification. For the rest of this research, the balanced train and test datasets will be used.

#### Classify text using CNN Model

The Hate Speech Dataset paper provides the link for the code used for the CNN model. The code below was downloaded from https://github.com/dennybritz/cnn-text-classification-tf and https://github.com/lubiluk/cnn-sentence. 

#### Prepare the dataset
1) Tokenize the text into number sequences
2) Pad sequences to the same length (of the longest sequence)
3) One hot encode train and test labels

In [ ]:
# Code from notebook provided in https://github.com/lubiluk/cnn-sentence
# Build the vocabulary
max_document_length = max([len(x.split(" ")) for x in cleanPosts_cb.clean_message.to_list()])
# Tokenizer default filtering seems to be identical to original clean_str
tokenizer = Tokenizer()
# Fit vocabulary using both train and test
tokenizer.fit_on_texts(cleanPosts_cb.clean_message.to_list())

# Convert train to sequences
x_train_cnn = np.array(tokenizer.texts_to_sequences(cleanPosts_tr.clean_message.to_list()), dtype='object')
# Pad examples to the same length
x_train_cnn = keras.preprocessing.sequence.pad_sequences(x_train_cnn,
                                               value=0,
                                               padding='post',
                                               maxlen=max_document_length)

# Convert test to sequences
x_test_cnn = np.array(tokenizer.texts_to_sequences(cleanPosts_ts.clean_message.to_list()), dtype='object')
# Pad examples to the same length
x_test_cnn = keras.preprocessing.sequence.pad_sequences(x_test_cnn,
                                               value=0,
                                               padding='post',
                                               maxlen=max_document_length)

# one hot encode train labels
y_train_cnn = pd.get_dummies(cleanPosts_tr.label).astype('int32').values

# one hot encode test labels
y_test_cnn = pd.get_dummies(cleanPosts_ts.label).astype('int32').values

#### Set up hyperparameters

In [ ]:
# Percentage of the training data to use for validation
dev_sample_percentage = 0.1
# Dimensionality of character embedding (default: 128)
embedding_dim = 128
# Filter sizes (default: [3, 4, 5])
filter_sizes = [3, 4, 5]
# Number of filters per filter size (default: 128)
num_filters = 128
# L2 regularization lambda (default: 0.0) (unused)
l2_reg_lambda = 0.0
# Dropout keep probability (default: 0.5)
dropout_keep_prob = 0.5
# Batch Size (default: 64)
batch_size = 64
# Number of training epochs (default: 200)
num_epochs = 200

#### Build the model

In [ ]:
# Build the model
sequence_length = x_train_cnn.shape[1]
num_classes = y_train_cnn.shape[1]
vocab_size = len(tokenizer.word_index) + 1
embedding_size = embedding_dim

print("Sequence Length: {}".format(sequence_length))
print("Number of Classes: {}".format(num_classes))
print("Vocabulary Size: {}".format(vocab_size))
print("Embedding Size: {}".format(embedding_size))

inputs = keras.layers.Input(shape=(sequence_length,))
x = keras.layers.Embedding(vocab_size, embedding_size, input_length=sequence_length,
                          embeddings_initializer='random_uniform')(inputs)
x = keras.layers.Reshape((sequence_length, embedding_size, 1))(x)

filter_layers = []

for s in filter_sizes:
  f = keras.layers.Conv2D(
      num_filters, kernel_size=(s, embedding_size), strides=(1, 1),
      padding='valid', data_format='channels_last', activation='relu',
      use_bias=True, kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.01),
      bias_initializer=keras.initializers.Constant(value=0.1))(x)
  f = keras.layers.MaxPooling2D(pool_size=(sequence_length - s + 1, 1), strides=(1, 1), padding='valid', data_format='channels_last')(f)
  filter_layers.append(f)

# Combine all the pooled features
x = keras.layers.concatenate(filter_layers, axis=1)
x = keras.layers.Reshape((num_filters * len(filter_sizes), ))(x)

# Add dropout
x = keras.layers.Dropout(dropout_keep_prob)(x)

# Final (unnormalized) scores and predictions
predictions = keras.layers.Dense(num_classes, use_bias=True, 
                                 bias_initializer=keras.initializers.Constant(value=0.1),
                                 kernel_initializer=keras.initializers.glorot_uniform(seed=None),
                                 kernel_regularizer=keras.regularizers.l2(l2_reg_lambda), 
                                 bias_regularizer=keras.regularizers.l2(l2_reg_lambda),
                                 activation='softmax')(x)
  
model = keras.Model(inputs=inputs, outputs=predictions)
model.summary()

#### Prepare the training process

In [ ]:
# Specify loss function and optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

#### Train the CNN Model

In [ ]:
history = model.fit(x_train_cnn,
                    y_train_cnn,
                    epochs=num_epochs,
                    batch_size=batch_size,
                    validation_split=dev_sample_percentage,
                    verbose=1)